# This code fetches era5 monthly averaged reanalysis temperature data from cds from 1994 till 2024 for Africa and converts this temperature data to a 30-band geotiff file for visualization in QGIS

In [1]:
import cdsapi

In [2]:
import numpy as np

In [2]:
#set your CDS API key and url
url = "https://cds.climate.copernicus.eu/api/v2"
cdsapi_key = "275708:61653f8b-f5e5-4f25-8400-f7cd5d1a67b1"

In [3]:
c = cdsapi.Client(url = url, key = cdsapi_key )

In [4]:
c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'grib',
        'product_type': 'monthly_averaged_reanalysis',
        'variable': '2m_temperature',
        'year': list(range(1994, 2024)),
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
        'area': [40, -25, -35, 60],
    },
    'download.grib')

2024-01-21 05:55:27,956 INFO Welcome to the CDS
2024-01-21 05:55:27,961 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-01-21 05:55:28,456 INFO Request is completed
2024-01-21 05:55:28,458 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data2/adaptor.mars.internal-1705773126.0350935-9805-13-9e29fb9d-f311-4830-ae7f-43c2a8c8b47c.grib to download.grib (70.5M)
2024-01-21 05:58:21,233 ERROR Download interupted: HTTPSConnectionPool(host='download-0016.copernicus-climate.eu', port=443): Read timed out.
2024-01-21 05:58:21,238 ERROR Download incomplete, downloaded 5501952 byte(s) out of 73958400
2024-01-21 05:58:21,239 WARNING Sleeping 10 seconds
2024-01-21 05:58:31,256 WARNING Resuming download at byte 5501952
2024-01-21 06:01:15,131 INFO Download rate 208.3K/s                                                                    


Result(content_length=73958400,content_type=application/x-grib,location=https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data2/adaptor.mars.internal-1705773126.0350935-9805-13-9e29fb9d-f311-4830-ae7f-43c2a8c8b47c.grib)

In [1]:
import xarray as xr

C:\Users\user\anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
from osgeo import gdal, osr

In [4]:
import cfgrib

C:\Users\user\anaconda\lib\site-packages\gribapi\__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.23.0
  warnings.warn(


In [5]:
# Input GRIB file
input_grib_file = 'download.grib'

In [7]:
# Output GeoTIFF file
output_tiff_file = 'era5_annual_temperature.tif'

In [8]:
# Open the GRIB file using cfgrib
ds = cfgrib.open_dataset(input_grib_file)

In [9]:
# Calculate annual means
ds_annual = ds.resample(time='Y').mean(dim='time')

C:\Users\user\anaconda\lib\site-packages\xarray\core\groupby.py:534: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  index_grouper = pd.Grouper(


In [10]:
# Extract latitude, longitude, and temperature data
latitude = ds_annual.latitude.values
longitude = ds_annual.longitude.values
temperature = ds_annual.t2m.values

In [40]:
# Create a multiband GeoTIFF file
driver = gdal.GetDriverByName('GTiff')
tiff_dataset = driver.Create(output_tiff_file, len(longitude), len(latitude), 30, gdal.GDT_Float32)

In [42]:
# Set GeoTIFF metadata
tiff_dataset.SetGeoTransform((longitude.min(), np.abs(longitude[1] - longitude[0]), 0, latitude.max(), 0, -np.abs(latitude[1] - latitude[0])))

0

In [43]:
# Set projection to WGS 84 (EPSG:4326)
srs = osr.SpatialReference()
srs.ImportFromEPSG(4326)
tiff_dataset.SetProjection(srs.ExportToWkt())

0

In [45]:
# Write temperature data to each band
for i in range(30):
    tiff_band = tiff_dataset.GetRasterBand(i + 1)
    tiff_band.WriteArray(temperature[i, :, :])
    tiff_band.SetNoDataValue(np.nan)
    tiff_band.FlushCache()

In [46]:

# Close the datasets
ds.close()
tiff_dataset = None